In [59]:
from dwave.system.samplers import LeapHybridSampler, DWaveSampler
from dwave.system.composites import EmbeddingComposite
import numpy as np
from pyqubo import Array, Constraint, Placeholder, Binary

schedules = {
	"task1": ["09:00", "12:00"],
	"task2": ["10:30", "13:00"],
	"task3": ["11:30", "12:00"],
	"task4": ["12:30", "13:40"],
	"task5": ["13:00", "15:00"],
	"task6": ["14:30", "15:00"],
	"task7": ["15:30", "18:00"],
    "task8": ["16:00", "17:00"],
}

Ne = 5
Nt = len(schedules)


In [ ]:
def time2num(time: str):
    h, m = map(float, time.split(":"))
    return h + m / 60

def check_overlap(time_slot1, time_slot2):
    start1, end1 = map(time2num, time_slot1)
    start2, end2 = map(time2num, time_slot2)

    return start1 < end2 and start2 < end1

In [12]:
q = Array.create('q', shape=(N, T, M), vartype='BINARY')

In [18]:
H_cost = 0
for a in range(M):
    for b in range(M):
        for t in range(T - 1):
            for i in range(N):
                H_cost += (q[i][t][a] * q[i][t + 1][b] * B[a][b]) * A[a][t] - 1

In [25]:
H_goal = Constraint(sum([sum([-q[i][t][G] for t in range(T)]) for i in range(N)]), label='goal')
print(H_goal)

Constraint(label=goal,((Binary(q[1][4][3])*Num(-1.000000)+Binary(q[1][3][3])*Num(-1.000000)+Binary(q[1][2][3])*Num(-1.000000)+Binary(q[1][0][3])*Num(-1.000000)+Binary(q[1][1][3])*Num(-1.000000))+Binary(q[0][4][3])*Num(-1.000000)+Binary(q[0][3][3])*Num(-1.000000)+Binary(q[0][2][3])*Num(-1.000000)+Binary(q[0][0][3])*Num(-1.000000)+Binary(q[0][1][3])*Num(-1.000000)))


In [26]:
H_place = Constraint(sum([sum([sum([q[i][t][a] - 1 for a in range(M)]) for t in range(T)]) for i in range(N)]), label='place')
H_time = Constraint(sum([sum([sum([q[i][t][a] - 1 for t in range(T)]) for a in range(M)]) for i in range(N)]), label='time')
H_start = Constraint(sum([1-q[i][0][0] for i in range(N)]), label='start')
cost = H_cost + H_place + H_goal + H_time + H_start

In [27]:
model = cost.compile()
qubo, offset = model.to_qubo()

In [52]:
token = 'DEV-a2fef127495e74bc669c5c5c64a0de948ca7fcb7'
endpoint = 'https://cloud.dwavesys.com/sapi'
solver = 'hybrid_binary_quadratic_model_version2'
dw = LeapHybridSampler(endpoint=endpoint, token=token, solver=solver)
print(dw)

In [53]:
sample_set = dw.sample_qubo(qubo)

In [54]:
print(sample_set)

  q[0][0][0] q[0][0][1] q[0][0][2] q[0][0][3] ... q[1][4][3] energy num_oc.
0          0          0          0          0 ...          0    9.0       1
['BINARY', 1 rows, 1 samples, 40 variables]


In [55]:
r = sample_set.record
print(r)

[([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 9., 1)]


In [56]:
decoded_sample = model.decode_sample(sample_set.first.sample, vartype="BINARY")
decoded_sample

DecodedSample(decoded_subhs=[Constraint(start,energy=2.000000),Constraint(time,energy=-35.000000),Constraint(goal,energy=-1.000000),Constraint(place,energy=-35.000000)],sample={q[1][4][2]:0,q[1][4][1]:0,q[1][4][0]:0,q[1][2][2]:0,q[1][1][2]:0,q[1][1][1]:0,q[1][1][0]:1,q[1][3][0]:1,q[1][0][2]:0,q[0][2][0]:0,q[0][3][1]:0,q[1][3][1]:0,q[0][1][3]:0,q[1][4][3]:0,q[0][1][2]:0,q[0][1][0]:1,q[0][1][1]:0,q[0][0][3]:0,q[1][3][3]:0,q[0][0][1]:0,q[1][1][3]:0,q[0][0][0]:0,q[1][2][0]:0,q[0][4][3]:0,q[1][2][3]:0,q[1][2][1]:0,q[0][2][2]:0,q[0][2][3]:0,q[0][0][2]:0,q[0][3][0]:1,q[1][3][2]:0,q[0][4][1]:0,q[0][3][3]:1,q[0][4][0]:0,q[0][2][1]:0,q[0][4][2]:0,q[1][0][3]:0,q[0][3][2]:0,q[1][0][0]:0,q[1][0][1]:0})

In [58]:
s = {}
for t in range(T):
    s[t] = {}
    for a in range(M):
        s[t][a] = decoded_sample.array('q', (0, t, a))
s

{0: {0: 0, 1: 0, 2: 0, 3: 0},
 1: {0: 1, 1: 0, 2: 0, 3: 0},
 2: {0: 0, 1: 0, 2: 0, 3: 0},
 3: {0: 1, 1: 0, 2: 0, 3: 1},
 4: {0: 0, 1: 0, 2: 0, 3: 0}}

In [44]:
print(decoded_sample.constraints(only_broken=True))

{'goal': (False, -3.0), 'place': (False, -35.0), 'start': (False, 2.0), 'time': (False, -35.0)}
